In [1]:
import pandas
import gripql
conn = gripql.Connection("http://grip.compbio.ohsu.edu")
O = conn.graph("bmeg_rc1_2")

## Download gene expression values from TCGA-READ cohort and build matrix with submitter id as label

In [11]:
c = O.query().V("Project:TCGA-READ").in_("InProject").in_("SampleFor").as_("sample")
c = c.in_("AliquotFor").in_("GeneExpressionOf").as_("exp")
c = c.render( ["$sample._data.gdc_attributes.submitter_id", "$exp._data.values"])
data = {}
for row in c.execute(stream=True):
    data[row[0]] = row[1]
samples = pandas.DataFrame(data).transpose().fillna(0.0)

[INFO]	2019-02-21 16:19:41,059	177 results received in 32 seconds


In [12]:
samples

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000273478,ENSG00000273481,ENSG00000273483,ENSG00000273484,ENSG00000273485,ENSG00000273486,ENSG00000273487,ENSG00000273489,ENSG00000273492,ENSG00000273493
TCGA-AF-A56N-01A,121.814080,1.613753,135.967454,5.391310,8.373278,1.996317,7.199910,55.575866,17.897951,19.853400,...,7.140150,0.069209,0.550757,1.003240,0.747594,1.196780,0.160442,4.731900,0.597412,1.663440
TCGA-AF-6655-01A,115.784470,0.552086,110.593452,5.686240,7.299788,2.813131,17.707582,60.536116,33.645363,11.259500,...,4.965650,0.105303,0.072872,1.029790,0.000000,1.064500,0.112764,3.341310,0.179748,0.000000
TCGA-AG-3584-01A,68.075530,0.177565,58.753341,5.938664,5.918852,2.984705,5.540848,28.806477,34.899924,9.784530,...,4.498400,0.000000,0.102532,0.000000,0.000000,0.901103,0.089732,1.896570,0.484521,0.000000
TCGA-AG-4021-01A,76.315404,3.621759,48.649980,8.770929,8.975365,4.697758,7.847580,94.176738,50.346372,11.064330,...,4.255500,0.000000,0.458015,0.806308,0.818607,0.750019,0.185226,2.879850,0.620093,2.423230
TCGA-AG-3602-01A,149.292850,13.769800,68.210860,2.952312,3.954100,1.095040,3.543324,22.680905,18.312414,4.514790,...,0.519277,0.000000,0.000000,0.000000,0.000000,0.287972,0.000000,1.052160,0.224449,0.000000
TCGA-AG-4015-01A,144.383820,0.975803,84.371536,4.042990,7.246945,1.798625,3.200457,64.655274,33.939024,13.325270,...,3.036880,0.000000,0.105375,0.000000,0.000000,0.432556,0.097495,2.007890,0.165042,0.000000
TCGA-DC-6154-01A,198.705750,6.444404,171.776070,4.535430,3.963755,2.289482,6.329372,63.010030,36.118073,16.551090,...,6.216120,0.000000,0.665686,0.567684,0.098619,0.809993,0.187067,6.767170,0.322654,1.795420
TCGA-DY-A1DG-01A,91.135477,1.760750,75.030610,3.376852,4.242396,0.354336,0.621496,56.402025,29.027294,11.371744,...,4.314640,0.179060,1.571240,0.547679,0.288324,1.245690,0.187989,8.431610,0.536561,0.680010
TCGA-F5-6571-01A,68.264725,0.871633,88.958358,5.325567,3.774668,4.162093,9.376368,42.817256,27.002879,10.747420,...,3.255440,0.000000,0.265342,0.584759,0.000000,1.111280,0.224792,1.417780,0.197543,0.705917
TCGA-AG-3885-01A,23.977357,0.549999,16.907770,2.662918,2.422737,3.163749,1.962252,28.543314,18.314048,6.616620,...,1.495260,0.000000,0.000000,1.486250,0.000000,0.179899,0.056147,0.302175,0.586954,0.000000
